In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('data-input.csv')
print(df)

In [4]:
df.to_csv('data.csv')

In [ ]:
df.head()

Found how to do most of this from "https://realpython.com/python-data-cleaning-numpy-pandas/"

Just removing some of the columns that we dont really care about or have overlap with other columns.

In [ ]:
to_drop = ['keywords',
            'homepage',
            'production_countries',
            'spoken_languages',
            'tagline',
            'status',
            'original_title']

new_names =  {'original_language': 'language'}
df.drop(to_drop, inplace=True, axis=1)
df.rename(columns=new_names, inplace=True)

# There was a bunch of rows that had 0 runtime/revenue that were skewing all my data. hopefully this helps?
df = df[df.revenue != 0]
df = df[df.runtime != 0]
df = df[df.vote_count != 0]

df.describe()

In [ ]:
df.head()

In [ ]:
# took from here https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas
# getting year month day for future calculations potentially
import datetime

df['year'] = pd.DatetimeIndex(df['release_date']).year
df['month'] = pd.DatetimeIndex(df['release_date']).month
df['day'] = pd.DatetimeIndex(df['release_date']).day


# Changing language so I can use this in my model
# af = 0, cn = 1, da = 2, de = 3, el = 4, en = 5, es = 6, fa = 7, fr = 8, he = 9, hi = 10, 
# id = 11, is = 12, it = 13, ja = 14, ko = 15, nb = 16, nl = 17, no = 18, pl = 19, pt = 20, ro = 21, 
# ru = 22, te = 23, th = 24, vi = 25, xx = 26, zh = 27
df['language'] = df['language'].map({'af' : 0, 'cn' : 1, 'da' : 2, 'de' : 3, 'el' : 4, 'en' : 5,
                                     'es' : 6, 'fa' : 7, 'fr' : 8, 'he' : 9, 'hi' : 10, 'id' : 11,
                                     'is' : 12, 'it' : 13, 'ja' : 14, 'ko' : 15, 'nb' : 16, 'nl' : 17,
                                     'no' : 18, 'pl' : 19, 'pt' : 20, 'ro' : 21, 'ru' : 22, 'te' : 23,
                                     'th' : 24, 'vi' : 25, 'xx' : 26, 'zh' : 27})

df.head()

In [7]:
# I had a value in my csv in 1927 with a 93 million dollar budget. I looked it up and this was incorrect
# So I needed to clean this datapoint and remove it from the csv
df = df[df.year != 1927]

In [8]:
df.to_csv('data-cleaned.csv')

In [9]:
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

data = pd.read_csv(r'data-cleaned.csv')

In [ ]:
plot1 = sns.scatterplot(x="budget", y="revenue", data=data)

plt.show()

# wouldnt save the jpg when I did it same as below for some reason.
fig = plot1.get_figure()
fig.savefig("plot1.jpg")

In [ ]:
plot2 = sns.lineplot(x="year", y="budget", data=data)

plt.show

plt.savefig("plot2.jpg")

In [ ]:
plot3 = sns.displot(data['month'])

plt.show

plt.savefig("plot3.jpg")

In [ ]:
data.describe()

In [ ]:
import tensorflow as tf

tf.random.set_seed(1234)

# I dont think this is right but im dropping the columns I dont need now just so I can split the input/output values

to_drop = ['Unnamed: 0',
            'genres',
            'id',
            'overview',
            'production_companies',
            'release_date',
            'title']

data.drop(to_drop, inplace=True, axis=1)
data.head()


In [45]:
# Taken from https://stackoverflow.com/questions/43697240/how-can-i-split-a-dataset-from-a-csv-file-for-training-and-testing
train = data.sample(frac=0.8, random_state = np.random.RandomState())
test = data.loc[~data.index.isin(train.index)]

train.to_csv('main_train.csv', index=False)
test.to_csv('main_test.csv', index=False)

print("--Get data--")
y_test = test.month
x_test = test.drop('month', axis=1)

y_train = train.month
x_train = train.drop('month', axis=1)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

--Get data--
(2698, 9) (674, 9) (2698,) (674,)


In [46]:
normalize = tf.keras.layers.Normalization()
normalize.adapt(x_train)

In [64]:
print("--Make model--")
model = tf.keras.models.Sequential([
  normalize,
  tf.keras.layers.Dense(256, input_shape=(8,), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(13, activation = "softmax")
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
  initial_learning_rate, decay_steps=100000, decay_rate=0.9, staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("--Fit model--")
model.fit(x_train, y_train, epochs=50, verbose=2, batch_size = 20)

print("--Evaluate model--")
model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

--Make model--
--Fit model--
Epoch 1/50
135/135 - 1s - loss: 2.5574 - accuracy: 0.1316 - 1s/epoch - 9ms/step
Epoch 2/50
135/135 - 0s - loss: 2.5086 - accuracy: 0.1457 - 494ms/epoch - 4ms/step
Epoch 3/50
135/135 - 1s - loss: 2.4810 - accuracy: 0.1638 - 550ms/epoch - 4ms/step
Epoch 4/50
135/135 - 1s - loss: 2.4705 - accuracy: 0.1679 - 514ms/epoch - 4ms/step
Epoch 5/50
135/135 - 1s - loss: 2.4496 - accuracy: 0.1720 - 561ms/epoch - 4ms/step
Epoch 6/50
135/135 - 1s - loss: 2.4470 - accuracy: 0.1635 - 535ms/epoch - 4ms/step
Epoch 7/50
135/135 - 1s - loss: 2.4383 - accuracy: 0.1646 - 547ms/epoch - 4ms/step
Epoch 8/50
135/135 - 1s - loss: 2.4324 - accuracy: 0.1805 - 514ms/epoch - 4ms/step
Epoch 9/50
135/135 - 0s - loss: 2.4362 - accuracy: 0.1727 - 476ms/epoch - 4ms/step
Epoch 10/50
135/135 - 0s - loss: 2.4206 - accuracy: 0.1898 - 462ms/epoch - 3ms/step
Epoch 11/50
135/135 - 1s - loss: 2.4197 - accuracy: 0.1723 - 502ms/epoch - 4ms/step
Epoch 12/50
135/135 - 0s - loss: 2.4196 - accuracy: 0.1790 